In [1]:
file = open("conversation.txt","r")
text = file.read()
# Removing \n at the end
text = text.split('\n')[0]
print(text)


Dr: Hi Jill, we would like to discharge you tomorrow and want to go over everything you need to know when you go home. Does that sound good? Pt: Yes that sounds great Dr: To start off, let’s review your medications. The only change is we need to replace your Mevacor to Lipitor, 20 mg once a day in the morning. Drink two cups of water daily. This is for your cholesterol. Due to your recent increase in mucus congestion, we’d like for you to start doing chest physical therapy 4 times daily as we discussed earlier. You also want to make sure you drink lots of water and take pancreatic enzymes to help with digestion. We’d like for you to get an MRI at Women’s College Hospital next Monday at 1 pm. We also want you to come back for a follow up appointment in 3 months. If you have any questions please feel free to contact us. Pt: Thank you so much doctor


In [2]:
import boto3
client = boto3.client(service_name='comprehendmedical', region_name='us-east-2')
result = client.detect_entities(Text= text)

In [3]:
entities = result['Entities'];
for entity in entities:
    print('Entity:', entity['Text'],', Category:',entity['Category'],', Type:', entity['Type'],', Score:',entity['Score'])

Entity: Jill , Category: PROTECTED_HEALTH_INFORMATION , Type: NAME , Score: 0.9997410178184509
Entity: Mevacor , Category: MEDICATION , Type: BRAND_NAME , Score: 0.970876932144165
Entity: Lipitor , Category: MEDICATION , Type: BRAND_NAME , Score: 0.999839186668396
Entity: mucus congestion , Category: MEDICAL_CONDITION , Type: DX_NAME , Score: 0.7271867990493774
Entity: chest , Category: ANATOMY , Type: SYSTEM_ORGAN_SITE , Score: 0.47813159227371216
Entity: chest physical therapy , Category: TEST_TREATMENT_PROCEDURE , Type: TREATMENT_NAME , Score: 0.41972899436950684
Entity: pancreatic enzymes , Category: MEDICATION , Type: GENERIC_NAME , Score: 0.9384152889251709
Entity: MRI , Category: TEST_TREATMENT_PROCEDURE , Type: TEST_NAME , Score: 0.9790312647819519
Entity: Women’s College Hospital , Category: PROTECTED_HEALTH_INFORMATION , Type: ADDRESS , Score: 0.9981922507286072


In [4]:
result.keys()

dict_keys(['Entities', 'UnmappedAttributes', 'ResponseMetadata'])

In [5]:
result['Entities'][5]

{'Id': 0,
 'BeginOffset': 455,
 'EndOffset': 477,
 'Score': 0.41972899436950684,
 'Text': 'chest physical therapy',
 'Category': 'TEST_TREATMENT_PROCEDURE',
 'Type': 'TREATMENT_NAME',
 'Traits': []}

### Medications

In [6]:
for entity in entities:
    if(entity['Category']=='MEDICATION'):
        print("Medication:",entity['Text'])
        try:
            for at in entity['Attributes']:
                print(at['Type'],':',at['Text'])
        except:
            pass
        print('')

Medication: Mevacor
DOSAGE : 20 mg
FREQUENCY : once a day

Medication: Lipitor
DOSAGE : 20 mg
FREQUENCY : once a day

Medication: pancreatic enzymes



### Tests to get done

In [7]:
for entity in entities:
    if(entity['Category']=='TEST_TREATMENT_PROCEDURE'):
        if(entity['Type']=='TEST_NAME'):
            print("Test:",entity['Text'])
            print('')
        try:
            for at in entity['Attributes']:
                print(at['Type'],':',at['Text'])
        except:
            pass
        

Test: MRI



### Things to remember

In [8]:
for entity in entities:
    if(entity['Category']=='TEST_TREATMENT_PROCEDURE'):
        if(entity['Type']=='TREATMENT_NAME'):
            print("Treatment:",entity['Text'])
        try:
            for at in entity['Attributes']:
                print(at['Type'],':',at['Text'])
        except:
            pass
        print('')

Treatment: chest physical therapy




## Everything Together

In [9]:
import boto3
def get_summary(text):
    client = boto3.client(service_name='comprehendmedical', region_name='us-east-2')
    result = client.detect_entities(Text= text)
    entities = result['Entities']
    result_dict={}
    result_dict['medications']=get_medications(entities)
    result_dict['tests']=get_tests(entities)
    result_dict['things_to_remember']=get_things_to_remember(entities)
    return result_dict
def get_medications(entities):
    medication_dict = {}
    for entity in entities:
        if(entity['Category']=='MEDICATION'):
            medication_dict[entity['Text']] = {}
            try:
                for at in entity['Attributes']:
                    medication_dict[entity['Text']][at['Type']] = at['Text']
            except:
                pass
    return medication_dict

def get_tests(entities):
    tests_dict = {}
    for entity in entities:
        if(entity['Category']=='TEST_TREATMENT_PROCEDURE'):
            if(entity['Type']=='TEST_NAME'):
                tests_dict[entity['Text']]={} 
            try:
                for at in entity['Attributes']:
                    tests_dict[entity['Text']][at['Type']] = at['Text']
            except:
                pass
    return tests_dict

def get_things_to_remember(entities):
    remember_dict={}
    for entity in entities:
        if(entity['Category']=='TEST_TREATMENT_PROCEDURE'):
            if(entity['Type']=='TREATMENT_NAME'):
                remember_dict[entity['Text']]={}
            try:
                for at in entity['Attributes']:
                    remember_dict[entity['Text']][at['Type']] = at['Text']
            except:
                pass

    return remember_dict

In [10]:
print(get_summary(text))

{'medications': {'Mevacor': {'DOSAGE': '20 mg', 'FREQUENCY': 'once a day'}, 'Lipitor': {'DOSAGE': '20 mg', 'FREQUENCY': 'once a day'}, 'pancreatic enzymes': {}}, 'tests': {'MRI': {}}, 'things_to_remember': {'chest physical therapy': {}}}
